## Implementing Reflexion Agent

### Actor Agent

#### Schema

In [2]:
# Imports
from pydantic import BaseModel, Field
from typing import List

In [3]:
class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous.")

In [60]:
class AnswerQuestion(BaseModel):
    """Answer the questions"""
    answer: str = Field(description="250 word detailed answer to the question.")
    reflection: Reflection = Field(description="Your reflection on the initial answer.")
    queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current answer.")

#### Chains

In [61]:
#imports
from datetime import datetime
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputToolsParser, PydanticToolsParser
from langchain_core.messages import HumanMessage


In [ ]:
load_dotenv()
llm = ChatGroq(model="llama-3.3-70b-versatile")
llm=llm.bind_tools(tools=[AnswerQuestion],tool_choice="AnswerQuestion")

In [63]:
llm

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f4b0ffa6510>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f4b0ffa7290>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'AnswerQuestion', 'description': 'Answer the questions', 'parameters': {'properties': {'answer': {'description': '250 word detailed answer to the question.', 'type': 'string'}, 'reflection': {'properties': {'missing': {'description': 'Critique of what is missing.', 'type': 'string'}, 'superfluous': {'description': 'Critique of what is superfluous.', 'type': 'string'}}, 'required': ['missing', 'superfluous'], 'type': 'object'}, 'queries': {'description': '1-3 search queries for researching improvements to address the critique of your current answer.', 'items': {'type': 'string'}, 'type': 'array'}}, 'required': ['answer', 'reflection', 'que

In [64]:
json_parser = JsonOutputToolsParser(return_id=True)
pydantic_parser = PydanticToolsParser(tools=[AnswerQuestion])

In [65]:
pydantic_parser

PydanticToolsParser(tools=[<class '__main__.AnswerQuestion'>])

In [66]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",""" 
                    You are expert researcher.
                    Current time: {time}

                    1. {first_instruction}
                    2. Reflect and critique your answer. Be severe to maximize improvement.
                    3. Recommend search queries to research information and improve your answer.
                 """),
        MessagesPlaceholder(variable_name="messages"),
        ("system","Answer the user's question above using the required format.")
    ]
).partial(time=lambda: datetime.now().isoformat())

In [67]:
first_responder_prompt_template = actor_prompt_template.partial(first_instruction="Provide a detailed 200 word answer.")

In [68]:
human_message = HumanMessage(
    content="Write about AI-Powered SOC / autonomous soc  problem domain, " \
    "list startups that do that and raised capital.")

In [69]:
chain = (first_responder_prompt_template|llm|pydantic_parser)

In [70]:
response = chain.invoke(input={"messages":[human_message]})

In [71]:
from pprint import pprint
pprint(response[0].answer)
pprint(response[0].reflection)
pprint(response[0].queries)

('An AI-Powered Security Operations Center (SOC) utilizes artificial '
 'intelligence and machine learning to enhance the efficiency and '
 'effectiveness of security operations. The primary goal is to identify, '
 'analyze, and respond to security threats in real-time, minimizing the risk '
 'of breaches and attacks. Autonomous SOC solutions aim to automate many of '
 'the manual tasks associated with security monitoring, incident response, and '
 'threat hunting, allowing human security analysts to focus on higher-level '
 'tasks that require expertise and judgment. Startups in this domain, such as '
 'Cybereason, Securonix, and Deep Instinct, have raised significant capital to '
 'develop and deploy AI-powered SOC solutions. For instance, Cybereason raised '
 '$350 million in Series F funding, while Securonix raised $1 billion in '
 'Series F funding. Other notable startups include JASK, which was acquired by '
 'Sumo Logic, and Cardinality, which offers an autonomous threat hunting

In [73]:
class ReviseAnswer(AnswerQuestion):
    "Revise your original answer to your question."
    references: List[str] = Field(description="Citations motivating your updated answer.")

In [74]:
revise_instructions = """
    Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.

"""

In [75]:
revisor = actor_prompt_template.partial(first_instruction=revise_instructions) | llm.bind_tools(tools=[ReviseAnswer],tool_choice="ReviseAnswer")